<a href="https://colab.research.google.com/github/PearceStrickland/Momentum-Trading-Algo/blob/main/Momentum_Algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install git+https://github.com/backtrader2/backtrader

  Cloning https://github.com/backtrader2/backtrader to /tmp/pip-req-build-wnth58gw
  Running command git clone -q https://github.com/backtrader2/backtrader /tmp/pip-req-build-wnth58gw
  Created wheel for backtrader: filename=backtrader-1.9.76.123-py3-none-any.whl size=419488 sha256=def9a3f5d33a2a9812c16637b16f9faf8d6bed8f4885a0d26f058f1eea723fcb
  Stored in directory: /tmp/pip-ephem-wheel-cache-qon2s4t4/wheels/c5/d9/ed/07ba378f0314939bc1697e304d2c3d45fb9a9c09f3da94b2a3
Successfully built backtrader


In [2]:
pip install git+https://github.com/quantopian/pyfolio

  Cloning https://github.com/quantopian/pyfolio to /tmp/pip-req-build-cppe9me7
  Running command git clone -q https://github.com/quantopian/pyfolio /tmp/pip-req-build-cppe9me7
     |████████████████████████████████| 52 kB 1.1 MB/s 
  Created wheel for pyfolio: filename=pyfolio-0.9.2+75.g4b901f6-py3-none-any.whl size=75775 sha256=8f323748d81b086cb771ddc0920035cfbf09d4cb6d3b97c1e8f2a7160a91c692
  Stored in directory: /tmp/pip-ephem-wheel-cache-qhhvba6a/wheels/2d/aa/24/c99ed55ef37c69e33815248c1622cdb81b65ec753868004c28
  Created wheel for empyrical: filename=empyrical-0.5.5-py3-none-any.whl size=39777 sha256=3d0a35f126b44008c62005f5e7e8244f2f10a174647735cd2d2903396ce07c1b
  Stored in directory: /root/.cache/pip/wheels/d9/91/4b/654fcff57477efcf149eaca236da2fce991526cbab431bf312
Successfully built pyfolio empyrical


In [3]:
pip install yfinance

     |████████████████████████████████| 6.3 MB 7.5 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24109 sha256=a67ddf6fc020bb8f23b630c4ffb4855c165ddd5325d18054d53314b1bcd8899a
  Stored in directory: /root/.cache/pip/wheels/86/fe/9b/a4d3d78796b699e37065e5b6c27b75cff448ddb8b24943c288
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [4]:
import backtrader as bt
import pandas as pd
import pyfolio as pf
from datetime import datetime
import matplotlib
matplotlib.use('Agg')
from google.colab import files
import yfinance as yf

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


MACD cross strategy using ATR for stop loss


In [5]:
class MACDCross(bt.Strategy):
  params = (
      # Standard MACD parameters
      #12 period EMA
      ('macd1', 12),
      #26 period EMA
      ('macd2', 26),
      #9 period signal line
      ('macdsig', 9),
      #A calculated using 14 periods
      ('atrperiod', 14),
      #Amount of true range distance you want for stop loss. Higher number=lower stop loss
      ('atrdist', 2.0), 

  ) 
  def log(self, txt, dt=None):
      ''' 
      Summary: This function allows us to print out whats going on inside the strategy
      Parameters: 
        - txt: Text to print out
        - dt: date that strategy event occured
      '''
      dt = dt or self.datas[0].datetime.date(0)
      print('%s, %s' % (dt.isoformat(), txt))
  
  def __init__(self):
    self.dataclose=self.datas[0].close
    self.order = None
    self.buyprice = None
    self.buycomm = None
    self.rsi = bt.indicators.RSI(self.datas[0])
    self.macd = bt.indicators.MACD(self.data,
                                       period_me1=self.p.macd1,
                                       period_me2=self.p.macd2,
                                       period_signal=self.p.macdsig)
    #Crossover indicatpr if data 1 inputted croses over data 2 value signal is 1 and if data 2 crosses data 1 value is negative 1
    self.mcross = bt.indicators.CrossOver(self.macd.macd, self.macd.signal)
    #Stoploss indicator using ATR
    self.atr = bt.indicators.ATR(self.data, period=self.p.atrperiod)
  
  def notify_order(self, order):
    if order.status in [order.Submitted, order.Accepted]:
            return
    if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)
    # After order has been either accepted or rejected, set order to none
    self.order = None

  def notify_trade(self, trade):
        """
        Summary: Another fancy print statement, except this one will print out 
        the P/L of a complete trade 
        """
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))


  def next(self):
    if self.order:
            return
    if not self.position:  # not in the market
      if self.mcross[0] > 0.0:
        self.order = self.buy()
        stoprange = self.atr[0] * self.p.atrdist
        self.stopamount = self.data.close[0] - stoprange
    else:
      pclose = self.data.close[0]
      stopamount = self.stopamount
      if pclose < stopamount:
        self.close()  # stop met - get out
      else:
        pdist = self.atr[0] * self.p.atrdist
        # Update only if greater than
        self.stopamount = max(stopamount, pclose - pdist)
    




Broker Code

In [6]:
# CREATE SIMPLE BROKER AND ADD STRATEGY

# create a "Cerebro" engine instance, Cerebro is what simulates a broker
cerebro = bt.Cerebro()

# Set Strategy Paramerters
pv = 10000
stock="SPY"

# Create a data feed
data = bt.feeds.YahooFinanceData(dataname=stock,
                                 fromdate=datetime(2020, 1, 9),
                                 todate=datetime(2021, 10, 31))

cerebro.adddata(data)  # Add the data feed

cerebro.addstrategy(MACDCross)  # Add the trading strategy

cerebro.broker.setcash(pv) # Set our desired cash start

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

# Set the commission
cerebro.broker.setcommission(commission=0.00)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % pv)

# These lines execute your code, and run Pyfolio on your strategy
# Pyfolio gives you a bunch of metrics to assess your strategy performance
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
strat=cerebro.run()

#Plotting , uncomment to download nice chart of trades and strategy performance
cerebro.plot()[0][0].savefig('smaStrat.png', dpi=300)
#files.download('smaStrat.png')

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

pyfoliozer = strat[0].analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
simple_tear_sheet = pf.create_simple_tear_sheet(returns)

ts = pf.create_returns_tear_sheet(returns, return_fig=True)
ts.savefig('pyfolio_returns_tear_sheet.png')



Starting Portfolio Value: 10000.00
2020-03-27, BUY EXECUTED, Price: 247.67, Cost: 2476.70, Comm 0.00
2020-06-12, SELL EXECUTED, Price: 301.42, Cost: 2476.70, Comm 0.00
2020-06-12, OPERATION PROFIT, GROSS 537.50, NET 537.50
2020-07-09, BUY EXECUTED, Price: 311.19, Cost: 3111.90, Comm 0.00
2020-09-04, SELL EXECUTED, Price: 339.96, Cost: 3111.90, Comm 0.00
2020-09-04, OPERATION PROFIT, GROSS 287.70, NET 287.70
2020-10-02, BUY EXECUTED, Price: 327.09, Cost: 3270.90, Comm 0.00
2020-10-27, SELL EXECUTED, Price: 335.04, Cost: 3270.90, Comm 0.00
2020-10-27, OPERATION PROFIT, GROSS 79.50, NET 79.50
2020-11-06, BUY EXECUTED, Price: 345.07, Cost: 3450.70, Comm 0.00
2021-01-28, SELL EXECUTED, Price: 372.71, Cost: 3450.70, Comm 0.00
2021-01-28, OPERATION PROFIT, GROSS 276.40, NET 276.40
2021-02-08, BUY EXECUTED, Price: 385.50, Cost: 3855.00, Comm 0.00
2021-02-26, SELL EXECUTED, Price: 380.63, Cost: 3855.00, Comm 0.00
2021-02-26, OPERATION PROFIT, GROSS -48.70, NET -48.70
2021-03-12, BUY EXECUTED, P

<IPython.core.display.Javascript object>

Final Portfolio Value: 11477.60


Start date,2020-01-09
End date,2021-10-29
Total months,21
,Backtest
Annual return,7.895%
Cumulative returns,14.776%
Annual volatility,4.242%
Sharpe ratio,1.81
Calmar ratio,3.82
Stability,0.87
Max drawdown,-2.068%


<IPython.core.display.Javascript object>

Start date,2020-01-09
End date,2021-10-29
Total months,21
,Backtest
Annual return,7.895%
Cumulative returns,14.776%
Annual volatility,4.242%
Sharpe ratio,1.81
Calmar ratio,3.82
Stability,0.87
Max drawdown,-2.068%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,2.07,2020-06-08,2020-06-11,2020-08-05,43
1,1.94,2021-05-07,2021-10-12,2021-10-26,123
2,1.50,2020-03-30,2020-04-01,2020-04-06,6
3,1.43,2021-01-25,2021-03-24,2021-04-05,51
4,1.29,2020-04-17,2020-04-21,2020-04-27,7


<IPython.core.display.Javascript object>